<a href="https://colab.research.google.com/github/Sparshsing/ai_timelapse/blob/main/time_lapse_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generate smooth Time lapse from multiple sequencial images

In [1]:
!git clone https://github.com/Sparshsing/ai_timelapse.git

Cloning into 'ai_timelapse'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 85 (delta 18), reused 78 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 14.16 MiB | 9.08 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [2]:
# change directory to ai_timelapse
%cd ai_timelapse

/content/ai_timelapse


In [5]:
import os
os.getcwd()

'/content/ai_timelapse'

In [6]:
# check GPU (select T4 runtime in settings if error)
!nvidia-smi

Fri Jan 17 06:22:03 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 700.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Settings

In [10]:
INPUT_DIR = 'input_frames/shahrukh-khan'
OUTPUT_DIR = 'output_frames/shahrukh-khan'
REFERENCE_IMG_NAME = '2017_raees.jpg'  # reference image for face tilt and lighting

preprocess_faces = True
FIX_LIGHTING = False  # to make images have a similar color
equal_intervals = True  # whether to use equal no of frames between images or decide based on date

# applicable when equal_intervals is False
# desired gap between frames in seconds (difference between dates of frames)
# eg. set 3600 to generate 1 frame for every 1 hr gap in input images
min_frame_duration = 3600 * 24

In [2]:

MAX_DIM = 1080
# set no of interpolated frames between images
MAX_INTERPOLATED_FRAMES = 15  # 1 less than powers of 2
MODEL_PATH = "pretrained_models/film_net/Style/saved_model"
VALID_IMG_FORMATS = ('.png', '.jpg', '.jpeg', '.bmp')

In [3]:
if MAX_INTERPOLATED_FRAMES + 1 not in (2, 4, 8, 16, 32, 64, 128):
    raise ValueError("MAX_INTERPOLATED_FRAMES + 1 must be a power of 2")

In [4]:
import os
from pathlib import Path
import numpy as np
import tempfile
import tensorflow as tf
import mediapy
import cv2
from PIL import Image, ExifTags
import re

from natsort import natsorted
import shutil
from tqdm import tqdm
from datetime import datetime
import subprocess
import math
import mediapy as media
import sys
from typing import Generator, Iterable, List, Optional


from eval import interpolator, util
import fix_images

2025-02-10 21:48:26.401972: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-10 21:48:26.402015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-10 21:48:26.402689: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 21:48:26.406735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-10 21:48:27.176604: W tensorflow/compiler/tf2

Model file already exists


## Config

In [5]:
# Download the pretrained model
import gdown
os.makedirs('pretrained_models/film_net/Style/saved_model', exist_ok=True)

if os.path.exists('pretrained_models/film_net/Style/saved_model/saved_model.pb'):
    print('Model already downloaded')
else:
  folder_url = 'https://drive.google.com/drive/folders/1i9Go1YI2qiFWeT5QtywNFmYAA74bhXWj'
  gdown.download_folder(folder_url, output='pretrained_models/film_net/Style/saved_model', quiet=False)

Model already downloaded


## Plan

In [6]:
# 1. Preprocess images
# 2. for succesive pairs of images:
# 3.     generate interpolated frames
# 4.     add interpolated frames to the output directory
# 5. generate video from output directory

## Utility Functions

In [7]:

# create empty output directory
def create_empty_dir(dir_name):
  if os.path.exists(dir_name):
    shutil.rmtree(dir_name)
  os.makedirs(dir_name)


def resize_and_save(input_path, output_path, size):
  if input_path.lower().endswith(VALID_IMG_FORMATS):
      img = Image.open(input_path)
      img = img.resize(size, Image.Resampling.LANCZOS)
      img.save(output_path)
  else:
      raise ValueError(f'Unsupported file format: {input_path}')


def get_sorted_images(input_dir: str) -> Path:
    input_dir = Path(input_dir)
    image_files = [
        f.as_posix() for f in input_dir.iterdir()
        if f.is_file() and f.suffix.lower() in VALID_IMG_FORMATS
    ]
    
    # Sort files naturally
    image_files = natsorted(image_files)

    # image_files = natsorted([f for f in os.listdir(input_dir) if f.lower().endswith(VALID_IMG_FORMATS)])
    if not image_files:
        raise Exception("No images found in the directory.", input_dir)
    return image_files
    

# prepare input data
def prepare_images(input_dir, processed_input_dir, preprocess_faces, size, reference_img_name=None, fix_lighting=False):
    files = get_sorted_images(input_dir)
  
    # find the reference image
    if reference_img_name == '':
        reference_path = files[0]
    else:
        reference_path = None
        for f in files:
            image_path = Path(f)
            if image_path.name == reference_img_name:
                reference_path = image_path.with_name(reference_img_name)
                break
            elif str(image_path) == reference_img_name:
                reference_path = image_path
                break
            else:
                continue
        if reference_path is None:
            raise Exception('Reference image not found')
    
    print('reference_image', str(reference_path))

    for f in tqdm(files, desc='Preparing images'):
        output_file = (Path(processed_input_dir) / Path(f).name).as_posix()
        if preprocess_faces:
            fix_images.process_face_image(
                f,
                output_file,
                reference_path=str(reference_path),
                background_color=(255, 255, 255),  # White background
                save_bbox_preview=False,
                fix_lighting=fix_lighting
                )
        else:
            resize_and_save(f, output_file, size)


def get_new_size(image_path: str, max_dim: int = MAX_DIM) -> np.ndarray:
    """Resize the image so that the maximum dimension is `max_dim`."""
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    if h > w:
        new_h = max_dim
        new_w = int(w * new_h / h)
    else:
        new_w = max_dim
        new_h = int(h * new_w / w)
    return (new_w, new_h)


def string_to_timestamp(datetime_str):
    # Try yyyymmdd_hhmmss pattern
    pattern1 = r'(\d{8}_\d{6})'
    match = re.search(pattern1, datetime_str)
    if match:
        try:
            return datetime.strptime(match.group(1), "%Y%m%d_%H%M%S")
        except ValueError:
            pass

    # Try yyyymmdd pattern
    pattern2 = r'(\d{8})'
    match = re.search(pattern2, datetime_str)
    if match:
        try:
            return datetime.strptime(match.group(1), "%Y%m%d")
        except ValueError:
            pass

    raise ValueError("No valid date found in string. Expected format: YYYYMMDD or YYYYMMDD_HHMMSS")


def timestamp_to_string(timestamp):
    # Convert the timestamp number to a datetime object
    dt = datetime.fromtimestamp(timestamp)
    # Format the datetime object to the desired string format
    formatted_string = dt.strftime("%Y%m%d_%H%M%S")
    return formatted_string


def choose_evenly_spaced_timestamps(timestamps, k):
    # Ensure we have at least k timestamps
    if k > len(timestamps):
        raise ValueError("k cannot be greater than the number of timestamps")

    # Calculate the interval between timestamps
    n = len(timestamps)
    interval = (n - 1) / (k - 1)

    # Select the timestamps
    chosen_timestamps = []
    for i in range(k):
        index = round(i * interval)
        chosen_timestamps.append(timestamps[index])

    return chosen_timestamps


def save_video(frames, out_path):
    ffmpeg_path = util.get_ffmpeg_path()
    mediapy.set_ffmpeg(ffmpeg_path)
    mediapy.write_video(out_path, frames, fps=30)


def save_frames(frames, output_dir, format='jpg'):
    """
    Save interpolated frames to the specified output directory and return the output paths.
    Args:
        frames: List of image arrays
        output_dir: Directory to save frames
        format: Image format to save (jpg/png)
    Returns:
        list: List of file paths where the frames are saved.
    """
    output_paths = []
    for idx, frame in enumerate(frames):
        output_path = os.path.join(output_dir, f'frame_{idx:06d}.{format}')
        util.write_image(output_path, frame)
        output_paths.append(output_path)
    return output_paths



def write_video_from_images(image_dir, output_path, fps):
    image_files = get_sorted_images(image_dir)
    
    # Read the first image to get the dimensions
    first_image_path = image_files[0]
    first_frame = cv2.imread(first_image_path)
    height, width, layers = first_frame.shape
    
    # Initialize the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    for image_file in image_files:
        frame = cv2.imread(image_file)
        if frame is None:
            print(f"Skipping {image_file}, cannot read image.")
            continue
        video_writer.write(frame)
    
    video_writer.release()
    print(f"Video saved to: {output_path}")

In [8]:

def interpolate_frames(image_1, image_2, times_to_interpolate, interpolator):
  input_frames = [str(image_1), str(image_2)]

  frames = list(
      util.interpolate_recursively_from_files(
          input_frames, times_to_interpolate, interpolator))
  return frames

## Main

In [9]:
# Preprocess images

processed_input_dir = tempfile.mkdtemp()
temp_interpolated_frames_dir = tempfile.mkdtemp()

files = get_sorted_images(INPUT_DIR)
for f in files:
    print(Path(f).name)


1992_deewana.jpg
1993_baazigar.jpg
1995_ddlj.jpg
1998_kuch_kuch_hota_hai.jpg
2001_kbkg.jpg
2003_kal_ho_na_ho.jpeg
2006_kabh_alvida.jpg
2009_billu.jpg
2011_don.jpg
2014_hny.jpg
2017_raees.jpg
2019_media.jpg
2023_pathan.jpg


In [11]:
new_size = get_new_size(files[0])
create_empty_dir(processed_input_dir)
prepare_images(INPUT_DIR, processed_input_dir, preprocess_faces, new_size, reference_img_name=REFERENCE_IMG_NAME, fix_lighting=FIX_LIGHTING)
files = get_sorted_images(processed_input_dir)


reference_image input_frames/shahrukh-khan/2017_raees.jpg


Preparing images:   0%|          | 0/13 [00:00<?, ?it/s]INFO:fix_images:Processing image: input_frames/shahrukh-khan/1992_deewana.jpg


INFO:fix_images:Image loaded successfully
libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1739204389.990509   17660 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
INFO:fix_images:Face detected successfully
INFO:fix_images:Image cropped and centered
INFO:fix_images:Image resized
INFO:fix_images:Background removed
libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No su

In [12]:
print('preprocessed images in', processed_input_dir)
print('file count', len(files))

preprocessed images in /tmp/tmp3235fmkm
file count 13


In [32]:
if not equal_intervals:
    for file in files:
      
      timestamp = string_to_timestamp(Path(file).stem)
      print(f"Filename: {file}, Datetime: {timestamp}")
else:
   for file in files:
      print(f"Filename: {file}")

Filename: 1992_deewana.jpg
Filename: 1993_baazigar.jpg
Filename: 1995_ddlj.jpg
Filename: 1998_kuch_kuch_hota_hai.jpg
Filename: 2001_kbkg.jpg
Filename: 2003_kal_ho_na_ho.jpeg
Filename: 2006_kabh_alvida.jpg
Filename: 2009_billu.jpg
Filename: 2011_don.jpg
Filename: 2014_hny.jpg
Filename: 2017_raees.jpg
Filename: 2019_media.jpg
Filename: 2023_pathan.jpg


In [ ]:
# Iterate through pairs of consecutive frames

interpolator_model = interpolator.Interpolator(MODEL_PATH, None)
print('Total frames:', len(files))
create_empty_dir(OUTPUT_DIR)
for i in range(len(files) - 1):
    print('processing frame', i)
    start_filename = files[i]
    end_filename = files[i + 1]

    if not equal_intervals:
        start_time = string_to_timestamp(Path(start_filename).stem)
        end_time = string_to_timestamp(Path(end_filename).stem)

    # Calculate times_to_interpolate for the required number of intermediate frames
    if equal_intervals:
          num_frames_needed = MAX_INTERPOLATED_FRAMES
    else:
          # Calculate the time difference in hours
          time_diff = (end_time - start_time).total_seconds()
          num_frames_needed = round(time_diff / min_frame_duration) - 1
          num_frames_needed = min(num_frames_needed, MAX_INTERPOLATED_FRAMES)
    times_to_interpolate = math.ceil(math.log2(num_frames_needed+1))  # n_intermediate = 2^k - 1

    frame_1 = start_filename
    frame_2 = end_filename
    output_frames = interpolate_frames(frame_1, frame_2, times_to_interpolate, interpolator_model)

    create_empty_dir(temp_interpolated_frames_dir)
    save_frames(output_frames, temp_interpolated_frames_dir)
    interpolated_files = get_sorted_images(temp_interpolated_frames_dir)

    if not equal_intervals:
        timestamps = np.linspace(start_time.timestamp(), end_time.timestamp(), len(interpolated_files))
        # find required frames from generated frames (can be extra)
        chosen_timestamps = choose_evenly_spaced_timestamps(timestamps, num_frames_needed+2)
        
        for filename, timestamp in zip(interpolated_files, timestamps):
            if timestamp not in chosen_timestamps:
                continue
            new_filename = Path(OUTPUT_DIR) / f"{timestamp_to_string(timestamp)}{Path(filename).suffix}"
            shutil.copyfile(filename, new_filename)
    else:
        for interp_index, filename in enumerate(interpolated_files[:-1]):
          new_filename = Path(OUTPUT_DIR) / f"{Path(start_filename).stem}_{interp_index:04d}{Path(filename).suffix}"
          shutil.copyfile(filename, new_filename)
          # include end frame in last iteration
          if i == len(files) - 2:
              new_filename = Path(processed_input_dir) / f"{Path(end_filename).stem}_0000{Path(filename).suffix}"
              shutil.copyfile(end_filename, new_filename)



2025-02-10 21:55:00.247774: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-10 21:55:00.260857: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-10 21:55:00.260974: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-10 21:55:00.263562: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-10 21:55:00.263610: I external/local_xla/xla/stream_executor

Total frames: 13
processing frame 0


100%|███████████████████████████████████████████████████████████████| 15/15 [01:02<00:00,  4.16s/it]


processing frame 1


100%|███████████████████████████████████████████████████████████████| 15/15 [00:13<00:00,  1.10it/s]


...

processing frame 10


100%|███████████████████████████████████████████████████████████████| 15/15 [00:13<00:00,  1.11it/s]


processing frame 11


100%|███████████████████████████████████████████████████████████████| 15/15 [00:13<00:00,  1.11it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmp3235fmkm/2023_pathan_0000.jpg'

In [15]:

# generate video from output directory
output_path = str(OUTPUT_DIR) + '.mp4'
fps = 24

write_video_from_images(OUTPUT_DIR, output_path, fps)

Video saved to: output_frames/shahrukh-khan.mp4


In [ ]:
# todo: try GAN using https://github.com/eladrich/pixel2style2pixel